# Aligning with minimap2


- alignment for short amplicons -- runtime: ~3min
```python
%%bash
minimap2="/v/scratch/tools/minimap2/minimap2"

REF="references/reference_short_oligos.fasta" 
READS_DIR="/v/volumes/nanopore/timin_uracil/oligos/timin20250417/timin_oligos_20250417/20250417_1447_MC-114328_AXB238_56275db9/fastq_pass"
OUT_DIR="/v/projects/nanopore/balazs/data/new_dataset/thymine_250417/short_alignments"

mkdir -p "$OUT_DIR"

for READ_FILE in "$READS_DIR"/*.fastq.gz; do
    BASENAME=$(basename "$READ_FILE" .fastq.gz)
    $minimap2 -a "$REF" "$READ_FILE" > "$OUT_DIR/${BASENAME}.sam"
done
```

- alignment for long amplicons -- runtime: ~17min
```python
%%bash
minimap2="/v/scratch/tools/minimap2/minimap2"

REF="references/reference_long_oligos.fasta" 
READS_DIR="/v/volumes/nanopore/timin_uracil/oligos/timin20250417/timin_oligos_20250417/20250417_1447_MC-114328_AXB238_56275db9/fastq_pass"
OUT_DIR="/v/projects/nanopore/balazs/data/new_dataset/thymine_250417/long_alignments"

mkdir -p "$OUT_DIR"

for READ_FILE in "$READS_DIR"/*.fastq.gz; do
    BASENAME=$(basename "$READ_FILE" .fastq.gz)
    $minimap2 -a "$REF" "$READ_FILE" > "$OUT_DIR/${BASENAME}.sam"
done
```

```python
%%bash
minimap2="/v/scratch/tools/minimap2/minimap2"

REF="references/reference_oligos.fasta" 
READS_DIR="/v/volumes/nanopore/timin_uracil/oligos/timin20250417/timin_oligos_20250417/20250417_1447_MC-114328_AXB238_56275db9/fastq_pass"
OUT_DIR="/v/projects/nanopore/balazs/data/new_dataset/thymine_250417/alignments"

mkdir -p "$OUT_DIR"

for READ_FILE in "$READS_DIR"/*.fastq.gz; do
    BASENAME=$(basename "$READ_FILE" .fastq.gz)
    $minimap2 -a "$REF" "$READ_FILE" > "$OUT_DIR/${BASENAME}.sam" 
done
```

In [ ]:
alignments_path = "/v/projects/nanopore/balazs/data/new_dataset/thymine_250417/alignments/*"
alignments_files = sorted(glob(alignments_path))

In [ ]:
sam_data = read_sam(alignments_files,
                          verbose=True,
                          has_movetable=False,
                          #min_MAPQ=30
                         )

In [ ]:
result = np.array([
    [read['contig_name'], read['map_quality']] for read in sam_data 
])

plt.hist(result[:,0], bins=500, label=f'{np.unique(result[:,0]).shape[0]}/500 reads')
plt.title('PCR dataset')
plt.xlabel('Sequence number')
plt.ylabel('Count')
plt.legend(loc='center right')
#plt.savefig('figures/pcr_histogram_minMQ=0.pdf')
plt.show()

---
# Exlopre aligned files


In [ ]:
from utils import *
from speedup import *
max_num_of_files = None

In [ ]:
short_alignments_path = "/v/projects/nanopore/balazs/data/new_dataset/thymine_250417/short_alignments/*"
short_alignments_files = sorted(glob(short_alignments_path))[:max_num_of_files]
long_alignments_path = "/v/projects/nanopore/balazs/data/new_dataset/thymine_250417/long_alignments/*"
long_alignments_files = sorted(glob(long_alignments_path))[:max_num_of_files]

In [ ]:
short_sam_data = read_sam(short_alignments_files,
                          verbose=True,
                          has_movetable=False,
                          #min_MAPQ=30
                         )

In [ ]:
long_sam_data = read_sam(long_alignments_files,
                         verbose=True,
                         has_movetable=False,
                         #min_MAPQ=30
                        )

In [ ]:
short_sam_read_ids = get_feature_from_sam_data(short_sam_data, 'read_id')
long_sam_read_ids = get_feature_from_sam_data(long_sam_data, 'read_id')


### Read sorting to `short` and `long` datasets

In [ ]:
set_short_sam_read_ids = set(short_sam_read_ids)
set_long_sam_read_ids = set(long_sam_read_ids)

reads_only_in_short_version = set_short_sam_read_ids - set_long_sam_read_ids
print("Reads in only PCR dataset:", len(reads_only_in_short_version))
reads_only_in_long_version = set_long_sam_read_ids - set_short_sam_read_ids
print("Reads in only Bacteria dataset:", len(reads_only_in_long_version))
reads_in_both_sets = set_short_sam_read_ids.intersection(set_long_sam_read_ids)
print("Reads in both datasets:", len(reads_in_both_sets))

In [ ]:
short_result = np.array([
    [read['contig_name'], read['map_quality']] for read in short_sam_data if read['read_id'] in reads_only_in_short_version
])

plt.hist(short_result[:,0], bins=500, label=f'{np.unique(short_result[:,0]).shape[0]}/500 reads')
plt.title('PCR dataset')
plt.xlabel('Sequence number')
plt.ylabel('Count')
plt.legend(loc='center right')
plt.savefig('figures/pcr_histogram_minMQ=0.pdf')
plt.show()

In [ ]:
long_result = np.array([
    [read['contig_name'], read['map_quality']] for read in long_sam_data if read['read_id'] in reads_only_in_long_version
])

x= plt.hist(long_result[:,0], bins=500, label=f'{np.unique(long_result[:,0]).shape[0]}/500 reads',  )

plt.title('Bacteria generated dataset')
plt.xlabel('Sequence number')
plt.ylabel('Count')
plt.legend(loc='center right')
plt.savefig('figures/bacteria_histogram_minMQ=0.pdf')
plt.show()

In [ ]:
x=np.histogram(long_result[:,0], bins=500)

In [ ]:
len(x[0])

In [ ]:
plt.scatter( x[1][:-1], x[0])

---
# Filtered reads by MapQual

In [ ]:
short_sam_data_filt = read_sam(short_alignments_files,
                          verbose=True,
                          has_movetable=False,
                          min_MAPQ=10
                         )

In [ ]:
long_sam_data_filt = read_sam(long_alignments_files,
                         verbose=True,
                         has_movetable=False,
                         min_MAPQ=10
                        )

In [ ]:
short_sam_read_ids_filt = get_feature_from_sam_data(short_sam_data_filt, 'read_id')
long_sam_read_ids_filt = get_feature_from_sam_data(long_sam_data_filt, 'read_id')

In [ ]:
set_short_sam_read_ids_filt = set(short_sam_read_ids_filt)
set_long_sam_read_ids_filt = set(long_sam_read_ids_filt)

reads_only_in_short_version_filt = set_short_sam_read_ids_filt - set_long_sam_read_ids_filt
print("Reads in only PCR dataset:", len(reads_only_in_short_version_filt))
reads_only_in_long_version_filt = set_long_sam_read_ids_filt - set_short_sam_read_ids_filt
print("Reads in only Bacteria dataset:", len(reads_only_in_long_version_filt))
reads_in_both_sets_filt = set_short_sam_read_ids_filt.intersection(set_long_sam_read_ids_filt)
print("Reads in both datasets:", len(reads_in_both_sets_filt))

In [ ]:
short_result_filt = np.array([
    [read['contig_name'], read['map_quality']] for read in short_sam_data_filt if read['read_id'] in reads_only_in_short_version_filt
])

plt.hist(short_result_filt[:,0], bins=500, label=f'{np.unique(short_result_filt[:,0]).shape[0]}/500 reads')
plt.title('PCR dataset')
plt.xlabel('Sequence number')
plt.ylabel('Count')
plt.legend(loc='center right')
plt.savefig('figures/pcr_histogram_minMQ=10.pdf')
plt.show()

In [ ]:
long_result_filt = np.array([
    [read['contig_name'], read['map_quality']] for read in long_sam_data_filt if read['read_id'] in reads_only_in_long_version_filt
])

plt.hist(long_result_filt[:,0], bins=500, label=f'{np.unique(long_result_filt[:,0]).shape[0]}/500 reads' )
plt.title('Bacteria generated dataset')
plt.xlabel('Sequence number')
plt.ylabel('Count')
plt.legend(loc='center right')
plt.savefig('figures/bacteria_histogram_minMQ=10.pdf')
plt.show()

----
# Phred scale / MapQual: (see [here](https://samtools.github.io/hts-specs/SAMv1.pdf) on p3.) 
Given a probability $0 < p ≤ 1$, the phred scale of
$$Phred =  \left\lfloor −10 \cdot \log_{10} (p) \right\rceil $$

In [ ]:
p = np.linspace(1e-6,1, 100000)
phred = np.round(-10*np.log10(p))
plt.plot(p, phred, '--', c='b')
plt.xlabel('probability that the base is incorrectly called')
plt.ylabel('PHRED score')
plt.savefig('figures/phred_score.pdf')
plt.grid()